In [1]:
# install psycopg2 in the terminal or run !pip install psycopg2
import psycopg2 as ps
import pandas as pd
import os

In [2]:
def connect_to_db(host_name, dbname, username, password, port):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
    return conn

In [3]:
# create table
def create_table(curr):
    create_table_command = """CREATE TABLE IF NOT EXISTS videos (
            video_id varchar(255) PRIMARY KEY,
            video_title TEXT NOT NULL,
            upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
            view_count INTEGER NOT NULL,
            like_count INTEGER NOT NULL,
            dislike_count INTEGER NOT NULL,
            favorite_count INTEGER NOT NULL,
            comment_count INTEGER NOT NULL
        ) """
    
    curr.execute(create_table_command)

In [4]:
#check to see if videos exist
def check_if_video_exists(curr, video_id):
    query = ("""SELECT video_id FROM videos WHERE video_id= %s""")
    curr.execute(query, (video_id,))

    a = curr.fetchone() is not None
    #return a single row from a table if a video is found with the id provided, if it doesn't it will return None
    return a 

In [5]:
#update row query
def update_row(curr, video_id, video_title, upload_date, view_count, like_count, dislike_count, favorite_count, comment_count):
    query = """
            UPDATE videos
            SET video_title = %s,
                upload_date = %s,
                view_count = %s,
                like_count = %s,
                dislike_count = %s,
                favorite_count = %s,
                comment_count = %s
            WHERE video_id = %s;
    """
    #the order of the vars need to be the same as the above query's, #video_id is for the lookup, don't confuse by the var name, it's to group what to be passed in the above query 
    vars_to_update = (video_title, upload_date, view_count, like_count, dislike_count, favorite_count, comment_count, video_id) 
    curr.execute(query, vars_to_update)

In [6]:
#updating the database
def update_table(curr, df):
    tmp_df=pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'dislike_count', 'favorite_count', 'comment_count'])
    
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']):
            update_row(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['dislike_count'], row['favorite_count'], row['comment_count'] )
        else:
            tmp_df = pd.concat([tmp_df, row.to_frame().T], ignore_index=True)
    
    tmp_df = tmp_df.reset_index(drop=True)
    
    return tmp_df


In [7]:
# insert query
def insert_video(curr, video_id, video_title, upload_date, view_count, like_count, dislike_count, favorite_count, comment_count):
    query = """ 
          INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count, dislike_count, favorite_count, comment_count)
          VALUES(%s, %s, %s, %s, %s, %s, %s, %s)  
    """
    vars_to_insert= (video_id, video_title, upload_date, view_count, like_count, dislike_count, favorite_count, comment_count)
    curr.execute(query, vars_to_insert)

In [8]:
# function to insert new video
def insert_videos_to_table(curr, df):
    for i, row in df.iterrows():
        insert_video(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['dislike_count'], row['favorite_count'], row['comment_count'])

In [9]:
#main
#keys
db_pass = os.environ.get('DBPW')


In [10]:
host_name='database-yt.cnyrgchewj4c.us-east-2.rds.amazonaws.com'
port = '5432'
dbname = 'postgres'   # not the database identifier, but postgres
username = 'liam'
password = db_pass
conn = None

conn = connect_to_db(host_name, dbname, username, password, port)

Connected!


In [11]:
#use the connection to create a cursor variable
curr = conn.cursor()

In [9]:
create_table(curr)


In [12]:
df = pd.read_csv("video_stats_new.csv", index_col=0)
df.head()

,video_id,video_title,upload_date,view_count,like_count,dislike_count,favorite_count,comment_count
0,d2neUrrUSu0,Python for Data Analysis: Probability And Bino...,2023-05-02,2200,5,1,2,3


In [13]:
new_vid_df = update_table(curr, df)

In [18]:
insert_videos_to_table(curr, new_vid_df)

In [19]:

conn.commit()

In [17]:
conn.rollback()